In [1]:
import pandas as pd

In [2]:
import os
os.getcwd()


'/Users/maximilianjoas/development/autoencodix_package/notebooks'

In [3]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Generate completely random data
def generate_random_data(n_samples=1000, n_features=400):
    """
    Generate random float data for testing neural networks
    
    Parameters:
    -----------
    n_samples : int, optional (default=1000)
        Number of samples in the dataset
    n_features : int, optional (default=400)
        Number of features per sample
    
    Returns:
    --------
    numpy.ndarray
        Random dataset of shape (n_samples, n_features)
    """
    return np.random.rand(n_samples, n_features).astype(np.float32)
data = generate_random_data()
data.shape

(1000, 400)

## Usage

In [4]:
# import module
import autoencodix as acx
# Use Vanillix Pipeline interface
# needs to be initialized with data
# data should be a numpy array, pandas dataframe or AnnData object
# possible to pass a custom Config object
van = acx.Vanillix(data=data)
# job of old make data
# populates self._features attrbute with torch tensor
# populates self._datasets attribute with torch dataset
# (important for training with dataloader)
# possible to pass a custom Config object, or keyword arguments
van.preprocess()
# job of old make model
# calls self.Trainer class to init and train model
# populates self._model attribute with trained model
# populates self.result attribute with training results (model, losses, etc)
van.fit()
# job of old make predict
# if no data is passed, used the test split from preprocessing
# otherwise, uses the data passed, and preprocesses it
# updates self.result attribute with predictions (latent space, reconstructions, etc)
van.predict()
# job of old make ml_task
# populates self.result attribute with ml task results
van.evaluate() # not implemented yet
# job of old make visualize
# populates self.result attribute with visualizations
van.visualize()
# show visualizations for notebook use
van.show_result()
# --------------------------
# --------------------------
# run all steps in the pipeline
result_object = van.run()



/Users/maximilianjoas/development/autoencodix_package/src/autoencodix/data/_numeric_dataset.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.data = torch.tensor(data, dtype=torch.float32)
Using bfloat16 Automatic Mixed Precision (AMP)


args and kwargs: (), {}
No user config found
default config: DefaultConfig(learning_rate=0.001, batch_size=32, epochs=100, latent_dim=16, n_layers=3, enc_factor=4, input_dim=10000, drop_p=0.1, weight_decay=0.01, use_gpu=False, n_devices=1, float_precision='bf16-mixed', gpu_strategy='auto', checkpoint_interval=10, reconstruction_loss='mse', default_vae_loss='kl', reproducible=True, global_seed=1)
Method config after updates: DefaultConfig(learning_rate=0.001, batch_size=32, epochs=100, latent_dim=16, n_layers=3, enc_factor=4, input_dim=10000, drop_p=0.1, weight_decay=0.01, use_gpu=False, n_devices=1, float_precision='bf16-mixed', gpu_strategy='auto', checkpoint_interval=10, reconstruction_loss='mse', default_vae_loss='kl', reproducible=True, global_seed=1)
args and kwargs: (), {}
No user config found
default config: DefaultConfig(learning_rate=0.001, batch_size=32, epochs=100, latent_dim=16, n_layers=3, enc_factor=4, input_dim=10000, drop_p=0.1, weight_decay=0.01, use_gpu=False, n_devic

TypeError: Expected value to be of type numpy.ndarray, got <class 'float'>.

In [ ]:
traindata = van._datasets.train
print(type(traindata))
print(type(traindata.data))
print(f"shape: {traindata.data.shape}")

<class 'autoencodix.data._numeric_dataset.NumericDataset'>
<class 'torch.Tensor'>
shape: torch.Size([700, 400])


In [ ]:
trainnloader = van._trainer._trainloader

In [ ]:
for i, data in enumerate(trainnloader):
    print(i, data.shape)
    break

0 torch.Size([32, 400])


## Detailed Usage Pipeline


In [ ]:

# import module
import autoencodix as acx
from autoencodix.utils.default_config import DefaultConfig
# Use Vanillix Pipeline interface
# needs to be initialized with data
# data should be a numpy array, pandas dataframe or AnnData object
# possible to pass a custom Config object
van = acx.Vanillix(data=sample_data)
# job of old make data
# populates self._features attrbute with torch tensor
# populates self._datasets attribute with torch dataset
# (important for training with dataloader)
# possible to pass a custom Config object, or keyword arguments
van.preprocess()
# job of old make model
# calls self.Trainer class to init and train model
# populates self._model attribute with trained model
# populates self.result attribute with training results (losses, etc)
van.fit()
""" 
Each step can be run separately, with custom parameters, these parameters
can be passed as keyword arguments, or as a Config object
"""
van.fit(learning_rate=0.01, batch_size=32, epochs=100) # or like this:
my_config = DefaultConfig(learning_rate=130.0, batch_size=32, epochs=100)
van.fit(config=my_config)



ModuleNotFoundError: No module named 'autoencodix.src'

In [ ]:
from autoencodix.data._numeric_dataset import NumericDataset
from autoencodix.base._base_dataset import BaseDataset
numeric = NumericDataset(sample_data)
base = BaseDataset(sample_data)

if isinstance(numeric, BaseDataset):
    print('numeric is instance of BaseDataset')

numeric is instance of BaseDataset


In [ ]:
import torch
from lightning_fabric import Fabric

In [ ]:
fabric = Fabric(accelerator="auto")
fabric.accelerator

### Documentation Config class